In [68]:
import requests
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
import time
import pandas as pd

In [100]:
webpage = requests.get('https://www.basketball-reference.com/boxscores/201810190ORL.html')

In [101]:
soup = BeautifulSoup(webpage.text, 'html.parser')

In [102]:
soup


<!DOCTYPE html>

<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201908281" rel="dns-prefetch"/>
<!-- no:cookie fast load the css.           -->
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://d2cwpp38twqe55.cloudfront.net" rel="preconnect"/>
<style>html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neue",helvetica,arial,sans-serif;color:#000}body{position:relative;background:#c9cbcd;z-index:0;-webkit-text-size-adjust:none;-moz-text-size-adjust:none;-ms-text-size-adjust:none}a img{border:0}ul,li,ol{margin:0;padding:0;list-style-type:none}table th,table td{border:0}iframe{max-width:100%}code{background

In [209]:
soup.findAll('tr', limit=2)

[<tr class="over_header"><th></th>
 <th aria-label="" class=" over_header center" colspan="20" data-stat="header_tmp">Basic Box Score Stats</th>
 </tr>, <tr>
 <th aria-label="Starters" class=" poptip sort_default_asc center" data-stat="player" scope="col">Starters</th>
 <th aria-label="Minutes Played" class=" poptip center" data-over-header="Basic Box Score Stats" data-stat="mp" data-tip="Minutes Played" scope="col">MP</th>
 <th aria-label="Field Goals" class=" poptip center" data-over-header="Basic Box Score Stats" data-stat="fg" data-tip="Field Goals" scope="col">FG</th>
 <th aria-label="Field Goal Attempts" class=" poptip center" data-over-header="Basic Box Score Stats" data-stat="fga" data-tip="Field Goal Attempts" scope="col">FGA</th>
 <th aria-label="Field Goal Percentage" class=" poptip center" data-over-header="Basic Box Score Stats" data-stat="fg_pct" data-tip="Field Goal Percentage" scope="col">FG%</th>
 <th aria-label="3-Point Field Goals" class=" poptip center" data-over-he

## Data I need to extract: 
FGM, 3P, FGA, FT, ORB, Opp DRB, TOV, FTA

In [253]:
headers_four_factors = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
# this allows me to get the header data from the html file - search the file for table rows, limit 2 and then return all table headers

In [176]:
headers_four_factors

['Starters',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 '+/-']

## This has everything except the opponents DRB but I can get that from the matchup

In [223]:
rows = soup.findAll('tr')[2:] # this pulls the rows data
player_stats1 = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
# the player_stats1 provides me all the data; however, the players name is not a table data but a table head so I need
# to create a separate dataframe and then join the two

In [227]:
rows[0].findAll('td')
# this is the first player in the box score for the day

[<td class="right " csk="1799" data-stat="mp">29:59</td>,
 <td class="right " data-stat="fg">6</td>,
 <td class="right " data-stat="fga">8</td>,
 <td class="right " data-stat="fg_pct">.750</td>,
 <td class="right " data-stat="fg3">3</td>,
 <td class="right " data-stat="fg3a">4</td>,
 <td class="right " data-stat="fg3_pct">.750</td>,
 <td class="right iz" data-stat="ft">0</td>,
 <td class="right iz" data-stat="fta">0</td>,
 <td class="right iz" data-stat="ft_pct"></td>,
 <td class="right iz" data-stat="orb">0</td>,
 <td class="right " data-stat="drb">6</td>,
 <td class="right " data-stat="trb">6</td>,
 <td class="right " data-stat="ast">5</td>,
 <td class="right " data-stat="stl">2</td>,
 <td class="right iz" data-stat="blk">0</td>,
 <td class="right " data-stat="tov">2</td>,
 <td class="right iz" data-stat="pf">0</td>,
 <td class="right " data-stat="pts">15</td>,
 <td class="right " data-stat="plus_minus">+32</td>]

In [229]:
rows[0].findAll('th')

<th class="left " csk="Batum,Nicolas" data-append-csv="batumni01" data-stat="player" scope="row"><a href="/players/b/batumni01.html">Nicolas Batum</a></th>

In [235]:
player_names1 = [[td.getText() for td in rows[i].findAll('th')] for i in range(len(rows))]
# this returns much more than just the name; instead it returns the entire table head flag

In [240]:
player_names1

[['Nicolas Batum'],
 ['Kemba Walker'],
 ['Jeremy Lamb'],
 ['Cody Zeller'],
 ['Marvin Williams'],
 ['Reserves',
  'MP',
  'FG',
  'FGA',
  'FG%',
  '3P',
  '3PA',
  '3P%',
  'FT',
  'FTA',
  'FT%',
  'ORB',
  'DRB',
  'TRB',
  'AST',
  'STL',
  'BLK',
  'TOV',
  'PF',
  'PTS',
  '+/-'],
 ['Malik Monk'],
 ['Michael Kidd-Gilchrist'],
 ['Willy Hernangómez'],
 ['Tony Parker'],
 ['Miles Bridges'],
 ['Dwayne Bacon'],
 ['Frank Kaminsky'],
 ['Bismack Biyombo'],
 ['Team Totals'],
 ['', 'Advanced Box Score Stats'],
 ['Starters',
  'MP',
  'TS%',
  'eFG%',
  '3PAr',
  'FTr',
  'ORB%',
  'DRB%',
  'TRB%',
  'AST%',
  'STL%',
  'BLK%',
  'TOV%',
  'USG%',
  'ORtg',
  'DRtg'],
 ['Nicolas Batum'],
 ['Kemba Walker'],
 ['Jeremy Lamb'],
 ['Cody Zeller'],
 ['Marvin Williams'],
 ['Reserves',
  'MP',
  'TS%',
  'eFG%',
  '3PAr',
  'FTr',
  'ORB%',
  'DRB%',
  'TRB%',
  'AST%',
  'STL%',
  'BLK%',
  'TOV%',
  'USG%',
  'ORtg',
  'DRtg'],
 ['Malik Monk'],
 ['Michael Kidd-Gilchrist'],
 ['Willy Hernangómez'],
 

In [245]:
test = list(zip(player_names1, player_stats1))

In [247]:
test[0:2]

[(['Nicolas Batum'],
  ['29:59',
   '6',
   '8',
   '.750',
   '3',
   '4',
   '.750',
   '0',
   '0',
   '',
   '0',
   '6',
   '6',
   '5',
   '2',
   '0',
   '2',
   '0',
   '15',
   '+32']),
 (['Kemba Walker'],
  ['26:46',
   '8',
   '16',
   '.500',
   '5',
   '10',
   '.500',
   '5',
   '5',
   '1.000',
   '0',
   '2',
   '2',
   '5',
   '1',
   '0',
   '3',
   '0',
   '26',
   '+34'])]

In [257]:
test2 = player_names1.append(player_stats1)

In [259]:
type(test2)

NoneType

In [251]:
stats_test = pd.DataFrame(test)

In [252]:
stats_test

,0,1
0,[Nicolas Batum],"[29:59, 6, 8, .750, 3, 4, .750, 0, 0, , 0, 6, ..."
1,[Kemba Walker],"[26:46, 8, 16, .500, 5, 10, .500, 5, 5, 1.000,..."
2,[Jeremy Lamb],"[25:05, 2, 7, .286, 0, 2, .000, 4, 4, 1.000, 1..."
3,[Cody Zeller],"[22:45, 3, 8, .375, 0, 0, , 2, 2, 1.000, 3, 5,..."
4,[Marvin Williams],"[20:29, 3, 7, .429, 2, 6, .333, 0, 0, , 1, 3, ..."
5,"[Reserves, MP, FG, FGA, FG%, 3P, 3PA, 3P%, FT,...",[]
6,[Malik Monk],"[23:37, 4, 12, .333, 2, 6, .333, 1, 1, 1.000, ..."
7,[Michael Kidd-Gilchrist],"[22:29, 5, 8, .625, 0, 2, .000, 2, 4, .500, 2,..."
8,[Willy Hernangómez],"[17:18, 2, 5, .400, 1, 1, 1.000, 1, 2, .500, 3..."
9,[Tony Parker],"[16:12, 0, 5, .000, 0, 1, .000, 0, 0, , 0, 3, ..."


In [249]:
stats = pd.DataFrame(player_stats1, columns = headers_four_factors[1:])

In [250]:
stats

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,29:59,6,8,.750,3,4,.750,0,0,,0,6,6,5,2,0,2,0,15,+32
1,26:46,8,16,.500,5,10,.500,5,5,1.000,0,2,2,5,1,0,3,0,26,+34
2,25:05,2,7,.286,0,2,.000,4,4,1.000,1,6,7,1,1,1,0,0,8,+19
3,22:45,3,8,.375,0,0,,2,2,1.000,3,5,8,2,1,0,0,2,8,+17
4,20:29,3,7,.429,2,6,.333,0,0,,1,3,4,0,0,0,1,0,8,+13
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,23:37,4,12,.333,2,6,.333,1,1,1.000,0,1,1,2,0,0,1,3,11,+5
7,22:29,5,8,.625,0,2,.000,2,4,.500,2,7,9,5,0,2,1,1,12,+24
8,17:18,2,5,.400,1,1,1.000,1,2,.500,3,2,5,2,1,0,2,4,6,+15
9,16:12,0,5,.000,0,1,.000,0,0,,0,3,3,6,0,0,2,2,0,+3


In [274]:
player = pd.DataFrame(player_names1)

In [275]:
player

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,Nicolas Batum,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Kemba Walker,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Jeremy Lamb,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Cody Zeller,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Marvin Williams,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,Reserves,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,None,None,None,None,None,None,None,None,None,None
6,Malik Monk,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,Michael Kidd-Gilchrist,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,Willy Hernangómez,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Tony Parker,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [278]:
player[0].shape

(68,)

In [277]:
stats.shape

(66, 20)

In [272]:
player_stats_total = 

TypeError: can't multiply sequence by non-int of type 'float'

In [58]:
year = 2019

In [59]:
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)

In [60]:
html = urlopen(url)

In [61]:
soup1 = BeautifulSoup(html)

In [62]:
soup1.findAll('tr', limit=2)

[<tr>
 <th aria-label="Rank" class="ranker poptip sort_default_asc show_partial_when_sorting center" data-stat="ranker" data-tip="Rank" scope="col">Rk</th>
 <th aria-label="Player" class=" poptip sort_default_asc center" data-stat="player" scope="col">Player</th>
 <th aria-label="Position" class=" poptip sort_default_asc center" data-stat="pos" data-tip="Position" scope="col">Pos</th>
 <th aria-label="Age of Player at the start of February 1st of that season." class=" poptip sort_default_asc center" data-stat="age" data-tip="Age of Player at the start of February 1st of that season." scope="col">Age</th>
 <th aria-label="Team" class=" poptip sort_default_asc center" data-stat="team_id" data-tip="Team" scope="col">Tm</th>
 <th aria-label="Games" class=" poptip center" data-stat="g" data-tip="Games" scope="col">G</th>
 <th aria-label="Games Started" class=" poptip center" data-stat="gs" data-tip="Games Started" scope="col">GS</th>
 <th aria-label="Minutes Played Per Game" class=" poptip 

In [64]:
headers = [th.getText() for th in soup1.findAll('tr', limit=2)[0].findAll('th')]

In [65]:
headers = headers[1:]
headers

['Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS']

In [66]:
rows = soup1.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [69]:
stats = pd.DataFrame(player_stats, columns = headers)

In [70]:
stats.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,.595,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,.576,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
5,Deng Adel,SF,21,CLE,19,3,10.2,0.6,1.9,.306,...,1.000,0.2,0.8,1.0,0.3,0.1,0.2,0.3,0.7,1.7
6,DeVaughn Akoon-Purcell,SG,25,DEN,7,0,3.1,0.4,1.4,.300,...,.500,0.1,0.4,0.6,0.9,0.3,0.0,0.3,0.6,1.0
7,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,.519,...,.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
8,Rawle Alkins,SG,21,CHI,10,1,12.0,1.3,3.9,.333,...,.667,1.1,1.5,2.6,1.3,0.1,0.0,0.8,0.7,3.7
9,Grayson Allen,SG,23,UTA,38,2,10.9,1.8,4.7,.376,...,.750,0.1,0.5,0.6,0.7,0.2,0.2,0.9,1.2,5.6


In [136]:
pwd

'/Users/fayadabbasi/Desktop/Python_Scripts/Galvanize/DSI'